In [9]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd

In [ ]:
episodes = []
for sn in range(1,7):
    response = requests.get('https://www.imdb.com/title/tt2575988/episodes?season='+str(sn))
    html_soup = BeautifulSoup(response.text, 'html5lib')
    episode_containers = html_soup.find_all('div', class_='info')
    for episode in episode_containers:
        season = sn
        episodeNum = episode.meta['content']
        episodeTitle = episode.a['title']
        airdate = episode.find('div', class_ = "airdate").text.strip()
        rating = episode.find('span', class_ = "ipl-rating-star__rating").text
        totalVotes = episode.find('span', class_ = "ipl-rating-star__total-votes").text
        desc = episode.find('div', class_ = 'item_description').text.strip()
        episode_data = [season, episodeNum, episodeTitle, airdate, rating, totalVotes, desc]
        episodes.append(episode_data)
episodes = pd.DataFrame(episodes, columns= ['season', 'episode_number', 'title', 'airdate', 'rating', 'total_votes', 'desc'])
episodes.head()

In [20]:
books = []
response = requests.get("https://www.goodreads.com/book/most_read")
html_soup = BeautifulSoup(response.text, 'html5lib')
book_container = html_soup.find_all('tr')
for book in book_container:
    bookTitle = book.find('a', class_ = "bookTitle").text.strip()
    bookAuthor = book.find('a', class_ = "authorName").text.strip()
    year = book.find('span', class_ = "greyText smallText uitext").text.strip()
    rating = book.find('span', class_ = "minirating").text.strip()
    readerNum = book.find('span', class_ = "greyText statistic").text.strip()
    book_data = [bookTitle, bookAuthor, year, rating, readerNum]
    books.append(book_data)
books = pd.DataFrame(books, columns=["Title", "Author", "Year", "Rating", "Readers"])
books.head()

,Title,Author,Year,Rating,Readers
0,Verity,Colleen Hoover,"4.42 avg rating — 1,223,712 ratings\n ...","4.42 avg rating — 1,223,712 ratings","11,552\n people read it"
1,I'm Glad My Mom Died,Jennette McCurdy,"4.66 avg rating — 122,041 ratings\n ...","4.66 avg rating — 122,041 ratings","10,405\n people read it"
2,It Ends with Us,Colleen Hoover,"4.38 avg rating — 1,650,787 ratings\n ...","4.38 avg rating — 1,650,787 ratings","8,343\n people read it"
3,Carrie Soto Is Back,Taylor Jenkins Reid,"4.32 avg rating — 64,942 ratings\n ...","4.32 avg rating — 64,942 ratings","7,919\n people read it"
4,The Seven Husbands of Evelyn Hugo,Taylor Jenkins Reid,"4.47 avg rating — 1,529,121 ratings\n ...","4.47 avg rating — 1,529,121 ratings","7,281\n people read it"


In [21]:
books.to_csv("US.csv", index = False)